In [1]:
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from sklearn.cluster import AffinityPropagation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df_A = pd.read_csv('AEP_daily.csv')
df_A ['Datetime']= pd.to_datetime(df_A['Datetime'])
df_A = df_A.set_index('Datetime')

In [3]:
def series_to_array(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1] #reshape to  list
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

<font size="16">Affinity Propagation</font><br> <br>
Para visualizar la agrupación de los datos introducidos, se utilizó el algoritmo de Affinity Porpagation por lo que es necesario definir el número de ventanas. Por favor, defínalo a continuación:

In [4]:
def affinity (v):
    print (f'El tamaño definido de la ventana es: {v}')
    x = series_to_array (df_A, v)
    df_A_copy = df_A.copy()
    df_A_values = df_A_copy.drop(df_A_copy.index[0:v],axis=0)
    
    AP = AffinityPropagation(random_state = 10).fit(x)
    n_clusters_ = len(AP.cluster_centers_indices_)
    AP_predict = AP.predict(x) 
    print (f'El número de agrupaciones determinado es: {n_clusters_}')
    plt.figure(figsize=(20,5))
    plt.scatter(df_A_values.index, df_A_values['AEP_MW'], c=AP_predict, cmap='tab20', s=7)
    plt.title('Consumo energético en el tiempo', fontweight="bold", fontsize=16)
    plt.xlabel('Tiempo', fontsize=12)
    plt.ylabel('Energía (MW)', fontsize=12)
    plt.show()
    
    scaler = StandardScaler()
    AEP_s = scaler.fit_transform(x)
    AEP_PCA = PCA(n_components = 2)
    AEP_PCA.fit(AEP_s)
    AEP_results = AEP_PCA.transform(AEP_s)
    x_PCA = pd.DataFrame(AEP_results)
    x_PCA.columns = ['One', 'Two']
    plt.figure(figsize=(20,5))
    plt.scatter(x_PCA['One'], x_PCA['Two'], c=AP_predict, cmap='tab20', s=7)
    plt.title('Vizualización de agrupación 2D', fontweight="bold", fontsize=16)
    plt.xlabel('Component 1', fontsize=12)
    plt.ylabel('Component 2', fontsize=12)
    plt.show()
    
widgets.interact(affinity, v=(1, 30, 1), k =(1, 15, 1))

interactive(children=(IntSlider(value=15, description='v', max=30, min=1), Output()), _dom_classes=('widget-in…

<function __main__.affinity(v)>